## First: ML Random Forest Classifier

As explained in the slides, some of the upcoming Python code blocks are adapted from my SLT assignment submission. In particular, the MIMIC IV, not the MIMIC III data is used.

In [1]:
# Import necessary libraries for the Random Forest Classifier (machine learning)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# In order to train the medication recommendation procedure
# we need to load in the MIMIC-IV demo diagnosis and prescription data
# './incomplete_mimic-iv-clinical-database-demo-2.2/hosp/' 
# is just where the csv files are stored
patients = pd.read_csv('./incomplete_mimic-iv-clinical-database-demo-2.2/hosp/patients.csv')
diagnoses = pd.read_csv('./incomplete_mimic-iv-clinical-database-demo-2.2/hosp/diagnoses_icd.csv')
prescriptions = pd.read_csv('./incomplete_mimic-iv-clinical-database-demo-2.2/hosp/prescriptions.csv')
print(patients.head())
print(diagnoses.head())
print(prescriptions.head())

   subject_id gender  anchor_age  anchor_year anchor_year_group         dod
0    10014729      F          21         2125       2011 - 2013         NaN
1    10003400      F          72         2134       2011 - 2013  2137-09-02
2    10002428      F          80         2155       2011 - 2013         NaN
3    10032725      F          38         2143       2011 - 2013  2143-03-30
4    10027445      F          48         2142       2011 - 2013  2146-02-09
   subject_id   hadm_id  seq_num icd_code  icd_version
0    10035185  22580999        3     4139            9
1    10035185  22580999       10     V707            9
2    10035185  22580999        1    41401            9
3    10035185  22580999        9     3899            9
4    10035185  22580999       11    V8532            9
   subject_id   hadm_id  pharmacy_id poe_id  poe_seq order_provider_id  \
0    10027602  28166872     27168639    NaN      NaN               NaN   
1    10027602  28166872     40720238    NaN      NaN              

In [3]:
# Patients are identified by their subject_id
# We will group the diagnoses and prescriptions by subject_id
# so that we can train a model to predict the medications
# based on the diagnoses
diagnoses_by_patient = diagnoses.groupby('subject_id')['icd_code'].apply(list).reset_index()
prescriptions_by_patient = prescriptions.groupby('subject_id')['drug'].apply(list).reset_index()
print(diagnoses_by_patient.head())
print(prescriptions_by_patient.head())

   subject_id                                           icd_code
0    10000032  [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...
1    10001217  [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...
2    10001725  [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...
3    10002428  [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...
4    10002495  [R310, N179, Z7901, Z86718, I214, A047, Z23, K...
   subject_id                                               drug
0    10000032  [Tiotropium Bromide, Tiotropium Bromide, Tiotr...
1    10001217  [Vial, Bag, OxycoDONE (Immediate Release) , Ox...
2    10001725  [Zolpidem Tartrate, Zolpidem Tartrate, Bag, Lo...
3    10002428  [Insulin, Insulin, Insulin, Multivitamins, Mul...
4    10002495  [Insulin, Insulin, Bag, Bag, Lidocaine Jelly 2...


In [4]:
# Pandas merge function is used to combine the two dataframes
# on the subject_id column
data = pd.merge(diagnoses_by_patient, prescriptions_by_patient, on='subject_id')
print(data.head())

   subject_id                                           icd_code  \
0    10000032  [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...   
1    10001217  [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...   
2    10001725  [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...   
3    10002428  [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...   
4    10002495  [R310, N179, Z7901, Z86718, I214, A047, Z23, K...   

                                                drug  
0  [Tiotropium Bromide, Tiotropium Bromide, Tiotr...  
1  [Vial, Bag, OxycoDONE (Immediate Release) , Ox...  
2  [Zolpidem Tartrate, Zolpidem Tartrate, Bag, Lo...  
3  [Insulin, Insulin, Insulin, Multivitamins, Mul...  
4  [Insulin, Insulin, Bag, Bag, Lidocaine Jelly 2...  


In [5]:
# In machine learning/statistics, we often use X to represent the features
# and y to represent the target variable
# So we intend to learn from the diagnoses (X) to predict the prescriptions (y)
X = data['icd_code']
y = data['drug']
print(X.head())
print(y.head())

0    [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...
1    [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...
2    [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...
3    [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...
4    [R310, N179, Z7901, Z86718, I214, A047, Z23, K...
Name: icd_code, dtype: object
0    [Tiotropium Bromide, Tiotropium Bromide, Tiotr...
1    [Vial, Bag, OxycoDONE (Immediate Release) , Ox...
2    [Zolpidem Tartrate, Zolpidem Tartrate, Bag, Lo...
3    [Insulin, Insulin, Insulin, Multivitamins, Mul...
4    [Insulin, Insulin, Bag, Bag, Lidocaine Jelly 2...
Name: drug, dtype: object


In [6]:
# We will now prepare the date for machine learning
# This is known as Feature Engineering
# MultiLabelBinarizer (mlb) helps us to convert the categorical diagnosis codes
# into a binary matrix (a matrix of 0's and 1's).
# This is necessary because machine learning models require numerical data
mlb_diagnosis = MultiLabelBinarizer()

In [7]:
# We will fit the MultiLabelBinarizer (mlb) on the diagnosis data
# and then transform the data into a binary matrix format
X_encoded = mlb_diagnosis.fit_transform(X)

In [8]:
# We will also use MultiLabelBinarizer (mlb) to convert the prescription data
# into a binary matrix format
# Below is the code from the previous two cells combined
# but for the prescription data
mlb_prescription = MultiLabelBinarizer()
y_encoded = mlb_prescription.fit_transform(y)

In [9]:
# Use the train_test_split method to split the data
# Here we are using 80% of the data for training and 20% for testing
# We also set a random_state so that the data is split in the same way each time
# This is useful for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=7)

In [10]:
# We'll now train the machine learning model
# Here, the model will learn patterns in the diagnoses data
# to predict the prescriptions
# The model will be a Random Forest (rf) Classifier, 
# which multiple decision trees to make predictions
# We will use 100 decision trees in the Random Forest
# We also set the random_state = 7 
# so that the model is trained in the same way each time
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=7)

In [11]:
# Here, the Random Forest Classifier is wrapped with in a MultiOutputClassifier
# The MultiOutputClassifier allows for multiple prescriptions to be predicted
# for each set of input patient diagnoses
multi_target_rf = MultiOutputClassifier(rf_classifier)
# Finally, fit() is a method that trains the model
# on the training data 
# X_train is the input data (patient diagnoses)
# y_train is the output data (prescription recommendations)
multi_target_rf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=7))

In [12]:
# The following recommend_medications() function takes in
# a list of diagnosis codes and returns the top 5 recommended medications
# based on the trained model

def recommend_medications(diagnosis_codes):
    # Here the square brackets enclosing the diagnosis_codes ensure that
    # the input is treated as a list of diagnosis codes
    # The transform() method converts the diagnosis codes into a binary format
    # that the model can understand
    patient_features = mlb_diagnosis.transform([diagnosis_codes])
    
    # predict_proba() is used to get the predicted probabilities
    # of each medication being recommended
    pred_probas = multi_target_rf.predict_proba(patient_features)
    
    # Get top 5 recommended medications names and return them
    top_5 = np.argsort(pred_probas[0])[::-1][:5]
    recommended_meds = mlb_prescription.classes_[top_5]
    
    return recommended_meds

In [13]:
# As an example, let us use the machine learning model to
# recommend medications given the following patient diagnoses
# 571.5: Cirrhosis of liver without mention of alcohol
# 250.00: Diabetes mellitus without mention of complication, 
# type II or unspecified type, not stated as uncontrolled
# Use the recommend_medications() function to get the recommended medication(s)
patient_diagnoses = ['571.5', '250.00']  
recommended_medications = recommend_medications(patient_diagnoses)
print("Predicted Drug:", recommended_medications)

C:\Users\Abrah\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['250.00', '571.5'] will be ignored
  warnings.warn(


Predicted Drug: [['0.45% Sodium Chloride' '*NF* Ertapenem Sodium']]


## Second: DL Recurrent Neural Network

In [14]:
# Load the required libraries, that we have not yet loaded
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

In [15]:
# Display the first few rows of each dataframe
# These dataframes were instantiated earlier in the notebook
print("Diagnoses Data:")
print(diagnoses.head())
print("\nPrescriptions Data:")
print(prescriptions.head())

Diagnoses Data:
   subject_id   hadm_id  seq_num icd_code  icd_version
0    10035185  22580999        3     4139            9
1    10035185  22580999       10     V707            9
2    10035185  22580999        1    41401            9
3    10035185  22580999        9     3899            9
4    10035185  22580999       11    V8532            9

Prescriptions Data:
   subject_id   hadm_id  pharmacy_id poe_id  poe_seq order_provider_id  \
0    10027602  28166872     27168639    NaN      NaN               NaN   
1    10027602  28166872     40720238    NaN      NaN               NaN   
2    10027602  28166872     62845687    NaN      NaN               NaN   
3    10027602  28166872     24340150    NaN      NaN               NaN   
4    10027602  28166872     14435820    NaN      NaN               NaN   

             starttime stoptime drug_type              drug  ...  gsn ndc  \
0  2201-10-30 12:00:00      NaN      MAIN  Fentanyl Citrate  ...  NaN NaN   
1  2201-10-30 12:00:00      NaN   

In [16]:
# Group diagnoses and prescriptions by patient and then merge the dataframes
diagnoses_grouped = diagnoses.groupby('subject_id')['icd_code'].apply(list).reset_index(name='icd_codes')
prescriptions_grouped = prescriptions.groupby('subject_id')['drug'].apply(list).reset_index(name='drugs')
merged_data = pd.merge(diagnoses_grouped, prescriptions_grouped, on='subject_id', how='inner')

# Display the merged data
print("\nMerged Data:")
print(merged_data.head())


Merged Data:
   subject_id                                          icd_codes  \
0    10000032  [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...   
1    10001217  [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...   
2    10001725  [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...   
3    10002428  [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...   
4    10002495  [R310, N179, Z7901, Z86718, I214, A047, Z23, K...   

                                               drugs  
0  [Tiotropium Bromide, Tiotropium Bromide, Tiotr...  
1  [Vial, Bag, OxycoDONE (Immediate Release) , Ox...  
2  [Zolpidem Tartrate, Zolpidem Tartrate, Bag, Lo...  
3  [Insulin, Insulin, Insulin, Multivitamins, Mul...  
4  [Insulin, Insulin, Bag, Bag, Lidocaine Jelly 2...  


In [17]:
# Convert ICD codes and drugs to strings
# Changes strings to lowercase
merged_data['icd_codes'] = merged_data['icd_codes'].apply(lambda x: [str(i) for i in x])
merged_data['drugs'] = merged_data['drugs'].apply(lambda x: [str(i).lower() for i in x])
print(merged_data.head())

   subject_id                                          icd_codes  \
0    10000032  [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...   
1    10001217  [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...   
2    10001725  [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...   
3    10002428  [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...   
4    10002495  [R310, N179, Z7901, Z86718, I214, A047, Z23, K...   

                                               drugs  
0  [tiotropium bromide, tiotropium bromide, tiotr...  
1  [vial, bag, oxycodone (immediate release) , ox...  
2  [zolpidem tartrate, zolpidem tartrate, bag, lo...  
3  [insulin, insulin, insulin, multivitamins, mul...  
4  [insulin, insulin, bag, bag, lidocaine jelly 2...  


In [18]:
# Create a vocabulary of unique ICD codes
icd_vocab = set()
for icd_list in merged_data['icd_codes']:
    icd_vocab.update(icd_list)
icd_vocab = list(icd_vocab)
print(icd_vocab)

['I132', 'E8120', 'H409', 'Z8571', '99811', '6253', 'A408', 'V1255', 'N319', 'V168', '1540', '42843', '3314', 'Z1611', '78791', 'T361X5A', '6826', 'N170', '2410', '20190', 'T8089XA', 'E8859', 'F1290', 'V4365', 'T501X5A', 'M4856XA', 'V1552', '7843', 'T17990A', 'S9032XA', 'E1152', 'E9320', 'Z515', 'V1011', '07070', 'J9610', 'D599', '6185', 'T370X5A', 'V1202', '9623', 'E8798', '99832', 'S72145A', '85221', '1759', '99674', 'Z8249', '2767', 'C3490', 'E60', 'V141', '42833', '04104', 'M720', 'R7881', 'E038', 'L97529', 'M25511', 'K5900', 'W1830XD', '44102', 'Z6826', 'Z9484', 'J159', '5939', 'I871', '2853', 'H53462', '8600', 'B182', '7904', 'I495', 'L0591', 'E887', 'S40022A', 'R112', 'Z86718', '3383', '3310', '2127', 'Z950', '5589', 'V431', 'S42291D', 'I4891', 'G931', '1977', 'E119', 'M19019', 'B9561', 'R45851', '49121', 'R634', 'E873', 'E872', '71596', 'I313', '75310', 'M545', 'M19012', '56789', 'I129', 'V6402', 'Z4502', '25080', '5849', 'Z961', 'K8020', 'E806', '1987', 'C3401', 'T383X1A', '55

In [19]:
# Create a vocabulary of unique drugs
drug_vocab = set()
for drug_list in merged_data['drugs']:
    drug_vocab.update(drug_list)
drug_vocab = list(drug_vocab)
print(drug_vocab)

['metamucil fiber singles', 'labetalol', 'multivitamin iv', 'phenobarbital ', 'ursodiol', 'ciprofloxacin 0.3% ophth soln', 'cytarabine', 'phenobarbital alcohol withdrawal dose taper (days 2-7)', 'desmopressin acetate', 'neostigmine', 'aspirin ec', 'brimonidine tartrate 0.15% ophth.', 'amino acids 5%-dextrose 15%', 'citrate dextrose 3% (acd-a) crrt', 'soln', 'bupivacaine 0.1%', 'dronabinol', 'lidocaine-prilocaine', 'digoxin', 'lorazepam', 'carvedilol', 'prednisone', 'lactulose enema', 'lidocaine 0.5%', 'benzonatate', 'amoxicillin-clavulanic acid', 'piperacillin-tazobactam', 'calcium gluconate sliding scale (critical care-ionized calcium)', 'prochlorperazine', 'ivermectin', 'tobramycin sulfate', 'metoprolol succinate xl', 'heparin flush (10 units/ml)', 'ferrous sulfate (liquid)', 'maalox/lidocaine', 'dorzolamide 2%/timolol 0.5% ophth.', 'neomycin-polymyxin-bacitracin', 'iso-osmotic dextrose', 'pregabalin', 'sw', 'glucose gel', 'ipratropium bromide mdi', 'dorzolamide 2% ophth. soln.', 'ro

In [20]:
# Count the number of unique ICD codes and drugs
print(f"Number of unique ICD codes: {len(icd_vocab)}")
print(f"Number of unique drugs: {len(drug_vocab)}")

Number of unique ICD codes: 1472
Number of unique drugs: 604


In [21]:
# Create mappings from ICD codes and drugs to indices
icd_to_index = {icd: idx for idx, icd in enumerate(icd_vocab)}
drug_to_index = {drug: idx for idx, drug in enumerate(drug_vocab)}
print(icd_to_index)
print(drug_to_index)

{'I132': 0, 'E8120': 1, 'H409': 2, 'Z8571': 3, '99811': 4, '6253': 5, 'A408': 6, 'V1255': 7, 'N319': 8, 'V168': 9, '1540': 10, '42843': 11, '3314': 12, 'Z1611': 13, '78791': 14, 'T361X5A': 15, '6826': 16, 'N170': 17, '2410': 18, '20190': 19, 'T8089XA': 20, 'E8859': 21, 'F1290': 22, 'V4365': 23, 'T501X5A': 24, 'M4856XA': 25, 'V1552': 26, '7843': 27, 'T17990A': 28, 'S9032XA': 29, 'E1152': 30, 'E9320': 31, 'Z515': 32, 'V1011': 33, '07070': 34, 'J9610': 35, 'D599': 36, '6185': 37, 'T370X5A': 38, 'V1202': 39, '9623': 40, 'E8798': 41, '99832': 42, 'S72145A': 43, '85221': 44, '1759': 45, '99674': 46, 'Z8249': 47, '2767': 48, 'C3490': 49, 'E60': 50, 'V141': 51, '42833': 52, '04104': 53, 'M720': 54, 'R7881': 55, 'E038': 56, 'L97529': 57, 'M25511': 58, 'K5900': 59, 'W1830XD': 60, '44102': 61, 'Z6826': 62, 'Z9484': 63, 'J159': 64, '5939': 65, 'I871': 66, '2853': 67, 'H53462': 68, '8600': 69, 'B182': 70, '7904': 71, 'I495': 72, 'L0591': 73, 'E887': 74, 'S40022A': 75, 'R112': 76, 'Z86718': 77, '338

In [22]:
# Function to convert lists of ICD codes and drugs to indices
def encode_data(data, icd_to_index, drug_to_index):
    encoded_icd = [icd_to_index[icd] for icd in data['icd_codes']]
    encoded_drugs = [drug_to_index[drug] for drug in data['drugs']]
    return encoded_icd, encoded_drugs

In [23]:
# Apply the encoding function
merged_data['encoded_icd'], merged_data['encoded_drugs'] = zip(*merged_data.apply(lambda row: encode_data(row, icd_to_index, drug_to_index), axis=1))

In [24]:
# Print the encoded data
print("\nEncoded Data:")
print(merged_data[['icd_codes', 'encoded_icd', 'drugs', 'encoded_drugs']].head())


Encoded Data:
                                           icd_codes  \
0  [2761, V08, 2875, 3051, 5715, 496, 07071, 7895...   
1  [V168, V161, 04109, 3485, 4019, 3484, 3240, 34...   
2  [49390, 7291, 6185, 56400, 5641, E9352, 6186, ...   
3  [4019, 29281, 4240, 00845, 1122, 7102, 73300, ...   
4  [R310, N179, Z7901, Z86718, I214, A047, Z23, K...   

                                         encoded_icd  \
0  [1264, 418, 1465, 954, 733, 895, 411, 1165, 89...   
1  [9, 1136, 678, 203, 492, 1469, 578, 1087, 277,...   
2  [320, 982, 37, 728, 1025, 379, 878, 432, 1223,...   
3  [492, 496, 1265, 258, 178, 234, 863, 831, 932,...   
4  [704, 405, 1468, 77, 1396, 1231, 956, 1075, 13...   

                                               drugs  \
0  [tiotropium bromide, tiotropium bromide, tiotr...   
1  [vial, bag, oxycodone (immediate release) , ox...   
2  [zolpidem tartrate, zolpidem tartrate, bag, lo...   
3  [insulin, insulin, insulin, multivitamins, mul...   
4  [insulin, insulin, bag, bag,

In [25]:
# Pad the encoded ICD sequences to a fixed length
def pad_sequence(sequence, max_length):
    if len(sequence) >= max_length:
        # Truncate if longer
        return sequence[:max_length]  
    else:
        # Pad with 0 if shorter
        return sequence + [0] * (max_length - len(sequence))  

In [26]:
# Pad or truncate each sequence
def pad_or_truncate(sequence, length):
    if len(sequence) > length:
        return sequence[:length]
    elif len(sequence) < length:
        return sequence + [0] * (length - len(sequence))
    return sequence

In [ ]:
# Define a maximum sequence length for ICD codes
MAX_ICD_LENGTH = 10
# Define the desired fixed length (e.g., 81)
fixed_length = 81
merged_data['encoded_drugs'] = merged_data['encoded_drugs'].apply(lambda x: pad_or_truncate(x, fixed_length))
merged_data['padded_icd'] = merged_data['encoded_icd'].apply(lambda x: pad_sequence(x, MAX_ICD_LENGTH))

In [29]:
# Convert data to PyTorch tensors
icd_tensor = torch.tensor(np.array(merged_data['padded_icd'].tolist()), dtype=torch.long)
drug_tensor = torch.tensor(merged_data['encoded_drugs'].tolist(), dtype=torch.long)

In [30]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(icd_tensor, drug_tensor, test_size=0.2, random_state=72)

In [31]:
# Is everything as expected?
print("\nTensor Shapes:")
print("ICD Tensor Shape:", icd_tensor.shape)
print("Drug Tensor Shape:", drug_tensor.shape)
print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)


Tensor Shapes:
ICD Tensor Shape: torch.Size([100, 10])
Drug Tensor Shape: torch.Size([100, 81])
X_train Shape: torch.Size([80, 10])
X_test Shape: torch.Size([20, 10])
y_train Shape: torch.Size([80, 81])
y_test Shape: torch.Size([20, 81])


In [32]:
class MedicationRecommender(nn.Module):
    def __init__(self, num_icd_codes, num_drugs, embedding_dim, hidden_dim):
        super(MedicationRecommender, self).__init__()
        self.embedding = nn.Embedding(num_icd_codes, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_drugs)

    def forward(self, icd_codes):
        embedded = self.embedding(icd_codes)
        lstm_out, _ = self.lstm(embedded)
        # Take the output from the last time step
        last_time_step = lstm_out[:, -1, :]
        output = self.linear(last_time_step)
        return output

In [33]:
# Model parameters
num_icd_codes = len(icd_vocab)
num_drugs = len(drug_vocab)
embedding_dim = 64
hidden_dim = 128

In [ ]:
# Instantiating the model
model = MedicationRecommender(num_icd_codes, num_drugs, embedding_dim, hidden_dim)

In [35]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    # Zero the gradients
    optimizer.zero_grad()  
    # Forward pass
    outputs = model(X_train)  
    # Calculate the loss
    loss = criterion(outputs, y_train)
    # Backpropagation
    loss.backward()  
    # Update weights
    optimizer.step()  

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

print('Finished Training')

In [ ]:
# Example input ICD codes (Cirrhosis of liver without mention of alcohol and Diabetes mellitus)
input_icd_codes = ['571.5', '250.00']
# Encode the input ICD codes
encoded_input = [icd_to_index[icd] for icd in input_icd_codes if icd in icd_to_index]
# Pad the encoded input
padded_input = pad_sequence(encoded_input, MAX_ICD_LENGTH)
# Convert to a PyTorch tensor
input_tensor = torch.tensor([padded_input], dtype=torch.long)

In [41]:
# Make a prediction
# Disable gradient calculation during inference
with torch.no_grad():  
    # Set the model to evaluation mode
    model.eval()  
    output = model(input_tensor)
    # Apply softmax to get probabilities
    probabilities = torch.softmax(output, dim=1)  
    # Get the index of the most likely drug
    predicted_class = torch.argmax(probabilities, dim=1).item()  

In [44]:
# Convert the predicted class index to the drug name
predicted_drug = drug_vocab[predicted_class]
# Print the predicted drug
print(f"Predicted Drug: {predicted_drug}")

Predicted Drug: propranolol la


## Third: ML vs DL Prediction Comparison

Using the Machine Learning Random Forest Classifier, we "0.45% Sodium Chloride" and "*NF* Ertapenem Sodium" were the predicted medications. Using the Deep Learning Recurrent Neural Network model, the predicted medications are "propranolol la". These predicted medications do not match. However, as the slides indicate, these medication recommendation systems should NOT be used in a clinical setting. Hence, perhaps the only reliable way to get informed feedback is to speak with an experienced physician, which I have not done.  